In [ ]:
# //https://medium.com/@researchgraph/what-is-phi-3-5-vision-101ce7c4d410

In [ ]:
import os
from PIL import Image 


from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

In [ ]:
import tensorflow as tf
import torch

print(tf.__version__) 
print(tf.config.list_physical_devices('GPU'))

print(torch.__version__)
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)

In [ ]:
model_id = "microsoft/Phi-3.5-vision-instruct" 

# Note: set _attn_implementation='eager' if you don't have flash_attn installed
model = AutoModelForCausalLM.from_pretrained(
  model_id, 
  device_map="cuda", 
  trust_remote_code=True, 
  torch_dtype="auto", 
  _attn_implementation='eager' # use _attn_implementation='eager' to disable flash attention , _attn_implementation='flash_attention_2'
)

# for best performance, use num_crops=4 for multi-frame, num_crops=16 for single-frame.
processor = AutoProcessor.from_pretrained(model_id, 
  trust_remote_code=True, 
  num_crops=16  # Changed to 16 for single-frame
) 

In [ ]:
images = []
placeholder = ""

In [ ]:
!pwd
! ls -la ./../../data/MultiModalGenAI/dog.png

In [ ]:
os.getcwd()

In [ ]:
local_image_path = "./../../data/MultiModalGenAI/dog.png"  # Replace with your actual image path
img = Image.open(local_image_path)
images.append(img)
placeholder += "<|image_1|>\n"
# url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png" 
# image = Image.open(requests.get(url, stream=True).raw) 

img.show()

In [ ]:
messages = [
    {"role": "user", "content": placeholder+"Describe this image in detail, including the main subject, actions, and setting."},
]

In [ ]:
prompt = processor.tokenizer.apply_chat_template(
  messages, 
  tokenize=False, 
  add_generation_prompt=True
)

inputs = processor(prompt, images, return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 1000,
    "temperature": 0.7,  # Added some randomness
    "do_sample": True,  # Enable sampling
    "top_k": 50,  # Added top-k sampling
} 

generate_ids = model.generate(**inputs, 
  eos_token_id=processor.tokenizer.eos_token_id, 
  **generation_args
)

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, 
  skip_special_tokens=True, 
  clean_up_tokenization_spaces=False)[0] 

print("\nDesribing the image:")
print(response)